# Optimizer Example

This notebook demonstrates how to use the PyQuOpt numerical optimization code to generate novel implementations of a specified unitary matrix. In this case, the Toffoli gate is explored.

### Imports

In [1]:
import sys
import numpy as np

sys.path.append('..')
from pyquopt import *

### Optimization Hyperparameters

In [2]:
num_qubits = 3
num_mq_instructions = 6 # want to explore circuits with six multi-qubit gates
num_params = 3 * num_qubits * (num_mq_instructions + 1)  # formula explained in MICRO paper

mq_dict = {
    0: ThreeGates.CR01,
    1: ThreeGates.CR02,
    2: ThreeGates.CV01,
    3: ThreeGates.CV02,
    4: ThreeGates.MCR,
}

In [3]:
alpha = 0.30  # penalty for non-standard angles
gamma = 1e-8  # penalty for large angles
non_fixed_params = np.ones(num_params)  # leave all parameters unfixed
fixed_params_vals = np.zeros(num_params)  # unfixed parameters should have a "fixed" value of 0

In [14]:
# mq_instructions = [0, 4, 4, 4, 0, 1]
mq_instructions = [4, 4, 4, 2, 3, 2]

### Instantiate and Run Optimizer

In [15]:
optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=ThreeGates.TOFFOLI,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)

In [16]:
opt_params, opt_val = optimizer.find_parameters_least_squares(20)

Note that there is also a function `optimizer.find_parameters_least_squares_par(num_guesses, num_procs)` that allows for process-based parallelism.

In [17]:
opt_val

0.6207950861000036

In [18]:
opt_params_rounded = round_params(opt_params)
print(opt_params_rounded)

[180. 135.  30.  80.  80.  25.  30.  60. 300. 270. 330. 120. 315. 260.
 300. 225. 260. 100. 315.  30. 260. 150. 265. 270. 180. 300. 120.  90.
 350.  10.  90. 240. 185. 225. 315. 150.   0. 330. 300. 315.  60. 260.
  45. 240. 240.   0. 225. 300.  85. 210. 150.  80. 315. 135.   0. 225.
 240. 315. 350. 170.  60.  45.  60.]


### Verify Implementation Accuracy

In [19]:
ub = UnitaryBuilder(num_qubits, mq_instructions, mq_dict)

In [20]:
implementation_matrix = ub.build_unitary(opt_params_rounded * np.pi / 180)

In [21]:
print(get_unitary_infidelity(ThreeGates.TOFFOLI, implementation_matrix, 8))

0.15694095494512406


An infidelity less than 0.01 is acceptable.